<h1 align="center"><img align="center" src="https://geoparse.io/graphics/geoparse_logo.png" alt="GeoParse Logo" width="200"/></h1>
<h1 align="center">GeoParse</h1>
<h3 align="center">All About Points <img src="https://geoparse.io/graphics/point.png" width="10"/> Lines <img src="https://geoparse.io/graphics/line.png" width="40"/> and Polygons <img src="https://geoparse.io/graphics/polygon.png" width="30"/></h3>

#### [HTML](http://geoparse.io/tutorials/spatial_index.html) 
***

# Geospatial Index

[![Binder](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gh/geoparse/geoparse/main?labpath=tutorials%2F00_visualization.ipynb)

This notebook demonstrates how to convert coordinates to geospatial indexes — Geohash, S2, and H3 — then convert them back to coordinates and measure the error.

***

In [ ]:
import os
import sys

sys.path.append(os.path.abspath("../geoparse/"))

In [ ]:
import warnings

import geopandas as gpd
import pandas as pd

from geoparse import SpatialIndex, plp

warnings.filterwarnings("ignore")

In [ ]:
# Reload all changed modules every time before executing a new line
%load_ext autoreload
%autoreload 2

## Points

In [ ]:
df = pd.read_csv("https://geoparse.io/tutorials/data/fatal_crash_great_britain_2023.csv")
df.head()

In [ ]:
len(df)

In [ ]:
plp(df)

In [ ]:
df["h3"] = SpatialIndex.ppoint_cell(df.latitude, df.longitude, cell_type="h3", res=15)
df.head()

In [ ]:
df[["lat", "lon"]] = SpatialIndex.pcell_point(df.h3, cell_type="h3")
df.head()

In [ ]:
max(abs(df.latitude - df.lat)), max(abs(df.longitude - df.lon))

In [ ]:
df["s2"] = SpatialIndex.ppoint_cell(df.latitude, df.longitude, cell_type="s2", res=30)
df.head()

In [ ]:
df[["lat", "lon"]] = SpatialIndex.pcell_point(df.s2, cell_type="s2")
df.head()

In [ ]:
max(abs(df.latitude - df.lat)), max(abs(df.longitude - df.lon))

In [ ]:
df["geohash"] = SpatialIndex.ppoint_cell(df.latitude, df.longitude, cell_type="geohash", res=10)
df.head()

In [ ]:
df[["lat", "lon"]] = SpatialIndex.pcell_point(df.geohash, cell_type="geohash")
df.head()

In [ ]:
max(abs(df.latitude - df.lat)), max(abs(df.longitude - df.lon))

# Polygons

In [ ]:
gdf = gpd.read_file("https://geoparse.io/tutorials/data/london.geojson")
gdf

In [ ]:
plp(gdf)

`plp` can visualize the cell layer over the polygon, but if you need the list of cells, the [ppoly_cell](https://geoparse.io/geoparse.html#geoparse.geoparse.SpatialIndex.ppoly_cell) (parallel polygon to cell converter) method from the `SpatialIndex` class takes a GeoDataFrame, cell type (h3, s2 or geohash) and resolution, and returns a tuple containing the list of cells and their counts.

In [ ]:
plp(gdf, h3_res=7)

In [ ]:
h3_list, h3_counts = SpatialIndex.ppoly_cell(gdf, cell_type="h3", res=7)

In [ ]:
h3_list[:10]

In [ ]:
h3_counts

When storing or transferring geospatial cell data, it's often more efficient to compact the cells, meaning you merge adjacent cells into their parent cells at a lower resolution. This reduces the total number of cells without losing spatial detail. 

In [ ]:
plp(gdf, cell_type="s2", s2_res=16, compact=True)

If `compact=True` is set, `ppoly_cell` returns a list of compacted cells, as shown in the figure above. However, it still reports the count of the original un-compacted cells. This means the reported count may be higher than the actual number of cells in the final (compacted) list. 

In [ ]:
s2_list, s2_counts = SpatialIndex.ppoly_cell(gdf, cell_type="s2", res=16, compact=True)
s2_counts

In [ ]:
len(s2_list)

If `verbose=True`, `ppoly_cell` prints progress updates and timing information to the console.

In [ ]:
s2_list, s2_counts = SpatialIndex.ppoly_cell(gdf, cell_type="s2", res=18, compact=True, verbose=True)

the `ppolycell` method can directly export cells to files. By default, `dump` is set to `None`, but if you provide a path (e.g., `dump='path/to/dir'`), it writes the cell IDs to multiple files in parallel.

In [ ]:
SpatialIndex.ppoly_cell(gdf, cell_type="h3", res=11, verbose=True, dump="./london")